# 1. Setup

In [ ]:
! pip install keras_tuner

In [ ]:
from tensorflow import keras
from keras import layers
from keras_tuner import  HyperModel
import tensorflow as tf
import os

In [ ]:
input_height = 69
input_width = 69
batch_size = 64

loss_function='categorical_crossentropy'
learning_rate= 1e-5
epochs = 100

In [ ]:
from keras import backend as K

def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

metrics = [
    precision,
    recall,
    tf.keras.metrics.CategoricalAccuracy(name='accuracy')
]

In [ ]:
paths = {
    'TRAIN_PATH' : os.path.join('workspace', 'images', 'train'),
    'TEST_PATH' : os.path.join('workspace', 'images','test'),
    'EVAL_PATH' : os.path.join('workspace', 'images','eval'),
    'LOG_DIR' : os.path.join('workspace', 'log_dir')
 }

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
train_dir=paths['TRAIN_PATH']
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_dir=paths['TEST_PATH']
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1./255) # it should not be augmented
validation_dir=paths['EVAL_PATH']
validation_generator = val_datagen.flow_from_directory(validation_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

In [ ]:
imgs_per_label = dict()
for i in range(9):
  path = os.path.join(paths['TRAIN_PATH'],str(i))
  n_images = len([f for f in os.listdir(path)if os.path.isfile(os.path.join(path, f))])
  imgs_per_label[i] = n_images

In [ ]:
# class weights computation
num_classes = 9
tot_images = sum(list(imgs_per_label.values()))
weights = dict([ (class_label , tot_images/(num_classes * n_images)) for class_label, n_images in imgs_per_label.items()])
print(weights)

# 2. Hyperparameters optimization

In [ ]:
import keras
import os

callbacks_list = [
        # interrupts training when accuracy has stopped improving accuracy on the validation set for at least 3+1=4 epochs
        keras.callbacks.EarlyStopping(
            monitor='val_loss', # should be part of the metrics specific during compilation
            patience=2,
        )
    ]

In [ ]:
import keras_tuner

# tune a model given an hypermodel, find the best hyperparameters, and return the tuned model
def tune_model(hypermodel):
    tuner = keras_tuner.BayesianOptimization(
        hypermodel,
        objective='val_loss',
        directory='galaxy_dir_bayes',
        project_name='galaxy_classification_project_bayes'
    )

    # search for the best hyperparameters
    tuner.search(train_generator, epochs=40, callbacks=callbacks_list, validation_data=validation_generator)
    # get the optimal hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    print(best_hps)
    tuner.results_summary()
    # tune the model with optimal parameters
    tuned_model = tuner.hypermodel.build(best_hps)
    return tuned_model

In [ ]:
class customNetwork(HyperModel):

    # override build
    def build(self,hp):
        model = keras.Sequential([
        layers.Input(shape=(69,69,3)),
        keras.layers.Conv2D(
            # filter and kernel size tuning
            filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
            activation='relu',
        ),
        layers.MaxPooling2D((2, 2),padding='same'),
        layers.BatchNormalization(),
        keras.layers.Conv2D(
            # filter and kernel size tuning
            filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
            kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
            activation='relu'
        ),
        layers.MaxPooling2D((2, 2),padding='same'),
        layers.BatchNormalization(),
        keras.layers.Flatten(),
        # dense layer tuning
        keras.layers.Dense(
            units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
            activation='relu'
        ),
        # dropout tuning
        layers.Dropout(rate=hp.Float(f"dropout_rate", min_value=0.3, max_value=0.7, step=0.2)),
        keras.layers.Dense(9, activation='softmax')
        ])
        model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=metrics)
        return model

    # override fit
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=64,
            **kwargs)

# 3. Save the optimal model

In [ ]:
model_cnn = tune_model(customNetwork())
model_cnn.save(os.path.join("model","tunedbayes_model.h"))